In [1]:
from datasets import load_dataset
import numpy as np
        
raw_dataset = load_dataset("csv", data_files="./../dataset/Rest_Mex_2022_Sentiment_Analysis_Track_Train.csv", delimiter=";")

Using custom data configuration default-13c2fcae2abc0c4d
Reusing dataset csv (/home/javilonso/.cache/huggingface/datasets/csv/default-13c2fcae2abc0c4d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
datapd = raw_dataset["train"].to_pandas()

In [3]:
print(f'Polarity values({datapd.Polarity.count()}):\n{datapd.Polarity.value_counts()}\n')
print(f'Atraction values({datapd.Attraction.count()}):\n{datapd.Attraction.value_counts()}\n')
print(f'Title values:\n{datapd.Title.str.len().describe()}\n')
print(f'Opinion values:\n{datapd.Opinion.str.len().describe()}') 

Polarity values(30212):
5    20936
4     5878
3     2121
2      730
1      547
Name: Polarity, dtype: int64

Atraction values(30212):
Hotel         16565
Restaurant     8450
Attractive     5197
Name: Attraction, dtype: int64

Title values:
count    30211.000000
mean        25.579987
std         16.099598
min          1.000000
25%         15.000000
50%         22.000000
75%         32.000000
max        153.000000
Name: Title, dtype: float64

Opinion values:
count    30210.000000
mean       625.505859
std        951.186641
min         25.000000
25%        213.000000
50%        309.000000
75%        615.000000
max      20438.000000
Name: Opinion, dtype: float64


In [4]:
# filter non empty title
datapd = datapd[datapd.Title.str.len() > 0]
# filter non empty opinion
datapd = datapd[datapd.Opinion.str.len() > 0]



In [5]:
# filter by num of characters of Opinion
datapd = datapd[datapd.Opinion.str.len() < 5000]
# datapd.Opinion.str.len().sort_values()

In [6]:
for i in range(5):
    datapd.loc[datapd.Polarity == i+1, 'Polarity'] = i
    
datapd.loc[datapd.Attraction == "Hotel", 'Attraction'] = 0
datapd.loc[datapd.Attraction == "Restaurant", 'Attraction'] = 1
datapd.loc[datapd.Attraction == "Attractive", 'Attraction'] = 2


In [7]:
datapd

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",0,0
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,0,1
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,0,0
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,0,0
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",0,0
...,...,...,...,...
30207,Verdadera joya arquitectónica,"Es una construcción majestuosa, creo que de la...",4,2
30208,Romántico,Muy al estilo de Romeo y Julieta es este sitio...,4,2
30209,Parece un castillo,Ideal para subir las escalinatas y divisar su ...,4,2
30210,Imperdible,"Es imperdible, de ahí puedes ver muy bien la c...",4,2


In [8]:
datapd.dtypes

Title         object
Opinion       object
Polarity       int64
Attraction    object
dtype: object

In [9]:
from datasets import Dataset
raw_dataset = Dataset.from_pandas(datapd)



In [10]:
print(raw_dataset)
raw_datasets = raw_dataset.shuffle(seed=50)
raw_datasets = raw_datasets.train_test_split(0.2)
print(raw_datasets)
raw_datasets.save_to_disk("./data")

Dataset({
    features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
    num_rows: 29931
})
DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 23944
    })
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 5987
    })
})


Flattening the indices:   0%|          | 0/24 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/6 [00:00<?, ?ba/s]

In [1]:
from datasets import load_dataset, DatasetDict
raw_dataset = DatasetDict.load_from_disk("./data")

In [3]:
raw_dataset.push_to_hub("javilonso/mex_data")


Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]